In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import sklearn.model_selection as skm
from ISLP import load_data, confusion_table
from ISLP.models import ModelSpec as MS
from sklearn.tree import (DecisionTreeClassifier as DTC,
                          DecisionTreeRegressor as DTR,
                          plot_tree,
                          export_text) #DTC for classification, DTR for regression, plot_tree to visualize tree, export_text to get rules
from sklearn.metrics import (accuracy_score,
                             log_loss) #accuracy_score for classification accuracy, log_loss for cross-entropy loss
from sklearn.ensemble import \
     (RandomForestRegressor as RF,
      GradientBoostingRegressor as GBR) #RF for random forest, GBR for boosting
from ISLP.bart import BART #Bayesian Additive Regression Trees
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Question 1

In [ ]:
# Load the Hitters dataset and remove rows with missing salary values
Hitters = load_data('Hitters')
Hitters = Hitters.dropna(subset=['Salary'])
Hitters

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,497,127,7,65,48,37,5,2703,806,32,379,311,138,N,E,325,9,3,700.0,N
318,492,136,5,76,50,94,12,5511,1511,39,897,451,875,A,E,313,381,20,875.0,A
319,475,126,3,61,43,52,6,1700,433,7,217,93,146,A,W,37,113,7,385.0,A
320,573,144,9,85,60,78,8,3198,857,97,470,420,332,A,E,1314,131,12,960.0,A


In [9]:
design = MS(Hitters.columns.drop('Salary')).fit(Hitters)
D = design.fit_transform(Hitters)
D = D.drop('intercept', axis=1) #drop intercept column
X = np.asarray(D)
y = np.array(Hitters['Salary'])
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
RFmodel_Hitters = RF(max_features=20, n_estimators=300, random_state=42)
RFmodel_Hitters.fit(X_train, y_train)
y_pred_RF_Hitters = RFmodel_Hitters.predict(X_test)
mse_RF_Hitters = mean_squared_error(y_test, y_pred_RF_Hitters)
print (f"Random Forest Model Test MSE: {mse_RF_Hitters:.2f}")

Random Forest Model Test MSE: 120962.79


Question 2